In [251]:
from pydantic import BaseModel, Field
import fitz
import pandas as pd
from tqdm import tqdm
from glob import glob
import openai
from pathlib import Path
import os
from openai import OpenAI
from datetime import datetime
import json
from enum import Enum

In [252]:
client = OpenAI(api_key='any-key', base_url='http://mn-dgx01.x5.ru:8000/v1')

Создадим pydentic-модель для получения структурированного вывода LLM

In [254]:
class ResumeJson(BaseModel):
    # graduation_year: int = Field(..., ge=1950, le=2050, description="Год окончания учебного заведения (от 1950 до 2050)")
    gpa: float = Field(..., ge=0, le=5, description="Средний балл GPA (от 0 до 5)")
    # certifications: bool = Field(..., description="Наличие сертификатов (True - есть, False - нет)")
    # has_honors_and_awards: bool = Field(..., description="Наличие наград и почетных званий (True - есть, False - нет)")
    cplus_plus_knowledge: bool = Field(..., description="Кандидат владеет C++ (True - да, False - нет)")
    csharp_knowledge: bool = Field(..., description="Кандидат владеет C# (True - да, False - нет)")
    sql_knowledge: bool = Field(..., description="Кандидат владеет SQL (True - да, False - нет)")
    python_knowledge: bool = Field(..., description="Кандидат владеет Python (True - да, False - нет)")
    javascript_knowledge: bool = Field(..., description="Кандидат владеет JavaScript (True - да, False - нет)")
    java_knowledge: bool = Field(..., description="Кандидат владеет Java (True - да, False - нет)")
    # data_analysis_skills: bool = Field(..., description="Навыки анализа данных (True - да, False - нет)")
    # project_management_skills: bool = Field(..., description="Навыки управления проектами (True - да, False - нет)")
    cloud_computing_skills: bool = Field(..., description="Навыки работы с облачными технологиями (True - да, False - нет)")
    # communication_skills: int = Field(..., ge=0, le=3, description="Коммуникативные навыки (0 - отсутствуют, 1 - низкие, 2 - средние, 3 - высокие)")
    # problem_solving_skills: int = Field(..., ge=0, le=3, description="Навыки решения проблем (0 - отсутствуют, 1 - низкие, 2 - средние, 3 - высокие)")
    teamwork_skills: int = Field(..., ge=0, le=3, description="Кандидат имеет навыки работы в команде (0 - отсутствуют, 1 - низкие, 2 - средние, 3 - высокие)")
    # leadership_skills: int = Field(..., ge=0, le=3, description="Лидерские навыки (0 - отсутствуют, 1 - низкие, 2 - средние, 3 - высокие)")
    english_knowledge: int = Field(..., ge=0, le=3, description="Знание английского языка (0 - отсутствует, 1 - базовый, 2 - средний, 3 - продвинутый)")
    # other_languages_count: int = Field(..., ge=0, le=10, description="Количество других иностранных языков")
    # technical_skills_count: int = Field(..., ge=0, le=100, description="Количество упомянутых технических навыков")
    # resume_length: int = Field(..., ge=1, le=5, description="Длина резюме (1-5 - количество страниц)")
    # formatting_quality: int = Field(..., ge=0, le=3, description="Качество форматирования резюме (0 - отсутствует, 1 - плохое, 2 - среднее, 3 - хорошее)")
    # grammar_and_spelling: bool = Field(..., description="Отсутствие грамматических и орфографических ошибок (True - да, False - нет)")
    # clear_objective: bool = Field(..., description="Наличие четко указанных целей (True - да, False - нет)")
    # cover_letter: bool = Field(..., description="Наличие сопроводительного письма (True - да, False - нет)")
    # references: bool = Field(..., description="Наличие рекомендаций (True - да, False - нет)")
    # number_of_projects: int = Field(..., ge=0, le=50, description="Количество выполненных проектов")
    # number_of_patents: int = Field(..., ge=0, le=20, description="Количество патентов")
    # number_of_publications: int = Field(..., ge=0, le=100, description="Количество публикаций")
    # number_of_awards: int = Field(..., ge=0, le=50, description="Количество наград")
    # has_open_source_contributions: bool = Field(..., description="Наличие вклада в open-source проекты (True - да, False - нет)")
    olympiad_participation: bool = Field(..., description="Кандидат участвовал в олимпиадах (True - да, False - нет)")
    # olympiad_wins: int = Field(..., ge=0, le=20, description="Количество побед в олимпиадах")
    # years_of_experience: int = Field(..., ge=0, le=600, description="Общий стаж работы кандидата в месяцах")
    # relevant_experience: float = Field(..., ge=0, le=50, description="Релевантный стаж работы (в годах)")
    # number_of_previous_jobs: int = Field(..., ge=0, le=20, description="Количество предыдущих мест работы")
    # duration_at_last_job: int = Field(..., ge=0, le=100, description=f"Продолжительность работы на последнем месте (в месяцах). Текущая дата: {datetime.now().strftime('%Y-%m-%d')}")
    # consistent_career_growth: bool = Field(..., description="Последовательный рост в карьере (True - да, False - нет)")
    # responsibilities_description_length: int = Field(..., ge=0, le=500, description="Длина описания обязанностей (в словах)")
    # achievements_quantified: bool = Field(..., description="Описание достижений в числовом выражении (True - да, False - нет)")
    has_previous_job_titles: bool = Field(..., description="У кандидата указаны прошлые должности (True - да, False - нет)")
    # has_management_experience: bool = Field(..., description="Наличие опыта управления (True - да, False - нет)")
    highest_education_level: int = Field(..., ge=1, le=3, description="Наивысший уровень образования (1 - среднее, 2 - бакалавр, 3 - магистр)")
    education_type: int = Field(..., ge=1, le=5, description="Тип образования (1 - программирование, 2 - математика, 3 - физика, 4 - экономика, 5 - другое)")
    university: int = Field(..., ge=1, le=14, description="""Одно из следующих учебных заведений:
                                1 - 'Московский государственный университет имени М.В. Ломоносова',
                                2 - 'Московский государственный технический университет имени Н.Э. Баумана',
                                3 - 'Московский физико-технический институт',
                                4 - 'Санкт-Петербургский государственный университет',
                                5 - 'Национальный исследовательский ядерный университет «МИФИ»',
                                6 - 'Национальный исследовательский университет "Высшая школа экономики"',
                                7 - 'Санкт-Петербургский политехнический университет Петра Великого',
                                8 - 'Национальный исследовательский Томский политехнический университет',
                                9 - 'Уральский федеральный университет имени первого Президента России Б.Н. Ельцина',
                                10 - 'Финансовый университет при Правительстве РФ',
                                11 - 'Университет ИТМО',
                                12 - 'Новосибирский национальный исследовательский государственный университет',
                                13 - 'Университет МИСИС'
                                14 - 'другое'""")
    experience_as_analyst: bool = Field(..., description="У кандидата имеется опыт работы в качестве аналитика (True - да, False - нет)")
    experience_as_data_scientist: bool = Field(..., description="У кандидата имеется опыт работы в качестве дата-сайентиста (True - да, False - нет)")
    experience_as_machine_learning_engineer: bool = Field(..., description="У кандидата имеется опыт работы в качестве инженера машинного обучения (True - да, False - нет)")
    experience_as_backend_developer: bool = Field(..., description="У кандидата имеется опыт работы в качестве бэкенд-разработчика (True - да, False - нет)")
    experience_as_frontend_developer: bool = Field(..., description="У кандидата имеется опыт работы в качестве фронтенд-разработчика (True - да, False - нет)")
    conference_participation: bool = Field(..., description="Кандидат участвовал в конференциях (True - да, False - нет)")
    hackathon_participation: bool = Field(..., description="Кандидат участвовал в хакатонах (True - да, False - нет)")
    # hackathon_wins: int = Field(..., ge=0, le=20, description="Количество побед в хакатонах")
    # candidate_level: int = Field(..., ge=1, le=4, description="Уровень кандидата (1 - intern, 2 - junior, 3 - middle, 4 - senior)")

In [255]:
def get_text_with_links(document):
    result_text = ""
    
    for page_num in range(len(document)):
        page = document.load_page(page_num)

        # Получаем слова и их координаты
        words = page.get_text("words")
        links = page.get_links()

        # Временный словарь для отслеживания последнего индекса слова для каждого URL
        last_occurrence_with_url = {}

        wc_map = {tuple(word[:4]): word[4] for word in words}
        
        for link in links:
            link_rect = fitz.Rect(link["from"])
            uri = link.get('uri', 'Нет ссылки')
            
            for i, word in enumerate(words):
                text = word[4]   # текст слова
                word_rect = fitz.Rect(word[:4])

                # Проверка на пересечение координат
                if word_rect.intersects(link_rect):
                    last_occurrence_with_url[uri] = (i, tuple(word[:4]))

        # Заменяем текст в конечном словаре ссылками на последних вхождениях
        for uri, (index, rect) in last_occurrence_with_url.items():
            wc_map[rect] = f'{wc_map[rect]}:"{uri}"'

        # Сортировка слов по их координатам для правильного порядка
        sorted_words = sorted(wc_map.items(), key=lambda x: (x[0][1], x[0][0]))

        for item in sorted_words:
            result_text += item[1] + " "
    
    return result_text.strip()

In [ ]:
# Загрузка данных
df = pd.read_csv('../../data/preprocessed/combined_data_preprocessed/kaiten_preprocessed/kaiten_preprocessed.csv')
df = df.reset_index(drop=True)
df = df.dropna()
# df = df.sort_values('Резюме')
df['text']=None
df['response']=None
df.head()

In [257]:
DATA_ROOT_PATH = Path("../../data/preprocessed/combined_data_preprocessed")

In [ ]:
for index, row in tqdm(df.iterrows()):
    resume = row["Резюме"]
    # resume = resume.replace("й", "й")
    
    if not resume.endswith(".pdf"):
        continue

    # pdf_path = os.path.join(DATA_ROOT_PATH, resume)
    pdf_path = DATA_ROOT_PATH / resume

    try:
        with fitz.open(pdf_path) as doc:
            text = get_text_with_links(doc)
    except Exception as e:
        print(resume.split("\\")[-1])

        print(f"Ошибка чтения файла: {e}")
        continue
    
    df.loc[index, ['text']] = [text]

    messages = [
        {"role": "system", "content": "Ты интеллектуальный ассистент для извлечения необходимой информации из резюме кандидата."},
        {'role': 'user', 'content': f"""**Задача:** 
        Проанализировать резюме кандидата и сформировать JSON

        **Требования**

        Внимательно читай описания к каждой из строк перед заполнением, обращай внимание на ограничения.
        В ответе предоставь только JSON.

        Резюме: {text}"""}
        ]

    try:
        response = client.chat.completions.create(
            model="x5-airun-medium",
            messages=messages,
            max_tokens=1024,
            temperature=0.2,
            top_p=0.95,
            n=1,
            extra_body={"guided_json": ResumeJson.model_json_schema()},
            stop=None,
        )
        # print(response.choices[0].message.content)
        df.loc[index, ['response']] = [response.choices[0].message.content]
    except Exception as e:
        print(f"Ошибка при обращении к API: {e}")
        continue

548it [1:11:56,  7.88s/it]


In [ ]:
df.head()

In [260]:
df.to_csv('x5_llm_features_kaiten.csv', index=False)